In [60]:
import numpy as np
import pandas as pd
from datetime import datetime

# Function to calculate ET0 using the Penman-Monteith model
def calculate_et0(row):
    # Extract meteorological parameters from the dataframe row
    T_max = row['TMAX']
    T_min = row['TMIN']
    RH_max = row['HMAX']
    RH_min = row['HMIN']
    solar_rad = row['ATOT']
    wind_speed = row['W2AVG']
    doy = row['DOY']
    latitude = 34.0442833  # Kashmir's l latitude
    altitude = 2166.5216352 # Kashmir's actual altitude 

    # Constants for calculations
    Cp = 0.001013
    epsilon = 0.622
    Lambda = 2.45

    # Calculate T_avg (average temperature)
    T_avg = (T_max + T_min) / 2

    # Calculate delta (slope of the saturation vapor pressure curve)
    delta = 4098 * (0.6108 * np.exp(17.27 * T_avg / (T_avg + 237.3))) / (T_avg + 237.3)**2

    # Calculate actual vapor pressure (e_actual) based on RH
    e_temp_max = 0.6108 * np.exp(17.27 * T_max / (T_max + 237.3))
    e_temp_min = 0.6108 * np.exp(17.27 * T_min / (T_min + 237.3))
    e_saturation = (e_temp_max + e_temp_min) / 2
    e_actual = (e_temp_min * (RH_max / 100) + e_temp_max * (RH_min / 100)) / 2

    # Calculate extraterrestrial radiation (Ra)
    dr = 1 + 0.033 * np.cos(2 * np.pi * doy / 365)
    phi = np.pi / 180 * latitude
    d = 0.409 * np.sin((2 * np.pi * doy / 365) - 1.39)
    omega = np.arccos(-np.tan(phi) * np.tan(d))
    Gsc = 0.0820
    Ra = (24 * 60 / np.pi) * Gsc * dr * (omega * np.sin(phi) * np.sin(d) + np.cos(phi) * np.cos(d) * np.sin(omega))

    # Calculate clear sky radiation (Rso)
    Rso = (0.75 + (2 * 10**-5) * altitude) * Ra

    # Calculate net solar radiation (Rn)
    alpha = 0.23
    Rns = (1 - alpha) * solar_rad
    sigma = 4.903 * 10**-9
    maxTempK = T_max + 273.16
    minTempK = T_min + 273.16
    Rnl = sigma * (maxTempK**4 + minTempK**4) / 2 * (0.34 - 0.14 * np.sqrt(e_actual)) * (1.35 * (solar_rad / Rso) - 0.35)
    Rn = Rns - Rnl

    # Soil heat flux density
    soil_heat_flux = 0

    # Calculate PET using Penman-Monteith equation
    gamma = (Cp * 101.3) / (epsilon * Lambda)
    PET = (0.408 * delta * (Rn - soil_heat_flux) + gamma * (900 / (T_avg + 273)) * wind_speed * (e_saturation - e_actual)) / (delta + gamma * (1 + 0.34 * wind_speed))

    return round(PET, 2)


         Date  DOY       TMAX       TMIN  HMAX   HMIN  ATOT     W2AVG   ET0
0  2005-01-01    1  21.161111  14.272222  97.5  65.97  4.09  5.194592  2.02
1  2005-01-02    2  21.261111   4.794444  99.3  77.37  4.11  3.428788  1.33
2  2005-01-03    3   5.855556   3.477778  99.8  98.20  2.98  3.249973  0.36
3  2005-01-04    4   4.644444   0.883333  99.6  98.50  1.21  3.527137  0.29
4  2005-01-05    5   0.827778  -9.172222  99.4  86.80  1.01  3.524851  0.37


In [69]:
data = {
    'Date': ['2005-01-01', '2005-01-02', '2005-01-03', '2005-01-04', '2005-01-05'],
    'DOY': [1, 2, 3, 4, 5],
    'TMAX': [21.161111, 21.261111, 5.855556, 4.644444, 0.827778],
    'TMIN': [14.272222, 4.794444, 3.477778, 0.883333, -9.172222],
    'HMAX': [97.5, 99.3, 99.8, 99.6, 99.4],
    'HMIN': [65.97, 77.37, 98.20, 98.50, 86.80],
    'ATOT': [4.09, 4.11, 2.98, 1.21, 1.01],  # Replace NaN with actual solar radiation values
    'W2AVG': [5.194592, 3.428788, 3.249973, 3.527137, 3.524851],  # Replace NaN with actual wind speed values
}

df = pd.DataFrame(data)

# Calculate ET0 for each row using the calculate_et0 function
df['ET0'] = df.apply(calculate_et0, axis=1)

# Print the resulting DataFrame with ET0 values
print(df)


         Date  DOY       TMAX       TMIN  HMAX   HMIN  ATOT     W2AVG   ET0
0  2005-01-01    1  21.161111  14.272222  97.5  65.97  4.09  5.194592  2.02
1  2005-01-02    2  21.261111   4.794444  99.3  77.37  4.11  3.428788  1.33
2  2005-01-03    3   5.855556   3.477778  99.8  98.20  2.98  3.249973  0.36
3  2005-01-04    4   4.644444   0.883333  99.6  98.50  1.21  3.527137  0.29
4  2005-01-05    5   0.827778  -9.172222  99.4  86.80  1.01  3.524851  0.37


In [61]:
dfk = pd.read_csv('Crop_kc.csv')

In [62]:
dfk

,Index,Crop,Kc ini,Kc mid,Kc end
0,1,Almonds,0.4,1.10,0.90
1,2,Apple,0.5,1.10,1.00
2,3,Apricot,0.5,1.10,1.00
3,4,Banana 1st year,0.5,1.10,1.00
4,5,Banana 2nd year,1.0,1.20,1.10
...,...,...,...,...,...
83,84,Walnut,0.5,1.10,1.00
84,85,Walnuts,0.4,1.10,0.90
85,86,Watermelon,0.4,1.00,0.75
86,87,Winter Wheat with frozen soils,0.4,1.15,0.30


In [63]:
dfk1 = dfk
dfk1['Crop'] = dfk['Crop'].str.lower()
dfk1.head()

,Index,Crop,Kc ini,Kc mid,Kc end
0,1,almonds,0.4,1.1,0.9
1,2,apple,0.5,1.1,1.0
2,3,apricot,0.5,1.1,1.0
3,4,banana 1st year,0.5,1.1,1.0
4,5,banana 2nd year,1.0,1.2,1.1


In [67]:

# Get user input for crop name and stage
crop_name = input("Enter the name of the crop: ")
stage = input("Enter the stage (ini, mid, or end): ")
crop_name = crop_name.lower()
stage = stage.lower()
# Check if the stage input is valid
valid_stages = ['ini', 'mid', 'end']
if stage not in valid_stages:
    print("Invalid stage input. Please enter 'ini', 'mid', or 'end'.")
else:
    # Filter the DataFrame based on user input
    filtered_df = dfk1[dfk1['Crop'] == crop_name]

    # Check if any rows match the filter criteria
    if not filtered_df.empty:
        # Get the Kc value based on the user input stage
        kc_value = filtered_df.iloc[0]['Kc ' + stage]  # Assuming column names are 'Kc ini', 'Kc mid', 'Kc end'
        print(f"Kc value for {crop_name} at {stage} stage is {kc_value}")
    else:
        print(f"No matching data found for {crop_name} at {stage} stage.")


Enter the name of the crop: apple
Enter the stage (ini, mid, or end): ini
Kc value for apple at ini stage is 0.5


In [68]:
day = int(input("Enter the day of the year: "))
selected_row = df[df['DOY'] == day]

if not selected_row.empty:
    et0 = selected_row['ET0'].values[0]
#     print(et0)
    
print(f"The evapotranspiration of the {crop_name} is {et0*kc_value}")

Enter the day of the year: 5
The evapotranspiration of the apple is 0.185


In [47]:
df

,Date,DOY,TMAX,TMIN,HMAX,HMIN,ATOT,W2AVG,ET0
0,2005-01-01,1,21.161111,14.272222,97.5,65.97,4.09,5.194592,2.20
1,2005-01-02,2,21.261111,4.794444,99.3,77.37,4.11,3.428788,1.52
2,2005-01-03,3,5.855556,3.477778,99.8,98.20,2.98,3.249973,0.46
3,2005-01-04,4,4.644444,0.883333,99.6,98.50,1.21,3.527137,0.32
4,2005-01-05,5,0.827778,-9.172222,99.4,86.80,1.01,3.524851,0.39
